# **Subclassing API** - *Inspiré de la formation de Thibault Neveu*

# 0. Imports

In [1]:
import tensorflow as tf
import numpy as np

# 1. Be sure to use Tensor Flow 2.0

In [2]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

# 2. Import the dataset

In [3]:
from sklearn.preprocessing import StandardScaler
# Fashion MNIST
fashion_mnist = tf.keras.datasets.fashion_mnist
(images, targets), (_, _) = fashion_mnist.load_data()
# Get only a subpart of the dataset
images = images[:10000]
targets = targets [:10000]
# Reshape the dataset and convert to float
images = images.reshape(-1, 784)
images = images.astype(float)
# Normalize images
scaler = StandardScaler()
images = scaler.fit_transform(images)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# 3. Create a model using the SubClassing API
The following call is not working because self.output is already set by the parent: tf.keras.Model

In [4]:
class CustomModel(tf.keras.Model):

    def __init__(self):
        super(CustomModel, self).__init__()

        # First in the init method you need to instanciate the layers you will use
        self.first_layer = tf.keras.layers.Dense(64)
        # WARNING: DO NOT CALL ONE OF YOUR CLASS VARIABLE: output
        self.output = tf.keras.layers.Dense(10, activation='softmax', name="d1")

    def call(self, x):
        # Then in the call method you can execute your operations
        layer1_out = self.first_layer(x)
        output = self.output_layer(layer1_out)
        return output

try:
    model = CustomModel()
except Exception as e:
    print("e=", e)

e= The layer custom_model has never been called and thus has no defined output.


In [5]:
class CustomModel(tf.keras.Model):

    def __init__(self):
        super(CustomModel, self).__init__()
        # First in the init method you need to instanciate the layers you will use
        self.first_layer = tf.keras.layers.Dense(64, activation="relu", name="first_layer")
        self.output_layer = tf.keras.layers.Dense(10, activation='softmax', name="output_layer")

    def call(self, x):
        # Then in the call method you can execute your operations
        prev = self.first_layer(x)
        out = self.output_layer(prev)
        return out

model = CustomModel()
model.predict(images[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step


array([[0.01554003, 0.41210625, 0.00205366, 0.01178397, 0.13008484,
        0.14418633, 0.11244058, 0.15012036, 0.01772731, 0.00395662]],
      dtype=float32)

In [6]:
print(targets[0:1])

[9]


The cell above is now working. We can call model.predict as we would do with a basic keras model. <br>
The <b>predict</b> method of the parent class called the <b>call</b> method from the <b>CustomModel</b> class.

In [7]:
model.predict(images[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


array([[0.01554003, 0.41210625, 0.00205366, 0.01178397, 0.13008484,
        0.14418633, 0.11244058, 0.15012036, 0.01772731, 0.00395662]],
      dtype=float32)

# 4. Train the model

The model can be used as a normal keras model. Thus, to train it, you just need to compile and to fit the model.

In [8]:
# Compile the model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

In [9]:
history = model.fit(images, targets, batch_size =1, epochs=10)

Epoch 1/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.7199 - loss: 0.8773
Epoch 2/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.8132 - loss: 0.5876
Epoch 3/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.8378 - loss: 0.4866
Epoch 4/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.8327 - loss: 0.5333
Epoch 5/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.8502 - loss: 0.5182
Epoch 6/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.8725 - loss: 0.4436
Epoch 7/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.8726 - loss: 0.4410
Epoch 8/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.8754 - loss: 0.5139
Epoch 9/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.8633 - loss: 0.6741
Epoch 10/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.8860 - loss: 0.4434


In [10]:
model.predict(images[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


array([[0.0000000e+00, 3.5191335e-34, 0.0000000e+00, 3.6359914e-38,
        7.2741993e-36, 0.0000000e+00, 0.0000000e+00, 3.0383053e-18,
        0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [11]:
print(targets[0:1])

[9]
